In [1]:
from CC import CC
from DnsVhost import DnsVhost
from MariaDB import MariaDB
from WP import WP
from Partition import Partition
from SMB import SMB
from NFS import NFS
from Diskquota import Diskquota
from ISCSI import ISCSI
from FTP import FTP
from RAID import RAID
from Tomcat import Tomcat

#---------------------------------------#
#             SSH 객체  생성             #
#---------------------------------------#
server_ip = "172.16.16.25"
server_account = "root"
server_passwd = "asd123!@"
cc =  CC(f"{server_ip}", f"{server_account}", f"{server_passwd}")
cc.connect()

#---------------------------------------#
#         여러 클래스 객체 생성            #
#---------------------------------------#
dv = DnsVhost("Rocky") 
mariadb = MariaDB()
wp = WP("Rocky")
part = Partition()
smb = SMB("Rocky")
nfs = NFS("Rocky")
dq = Diskquota("Rocky")
iscsi = ISCSI("Rocky","Rocky")
ftp = FTP("Rocky")
raid = RAID("Rocky")
tomcat = Tomcat("Rocky")

#---------------------------------------#
#                 전역변수               #
#---------------------------------------#
ZONEFILE = ""


###################################################
#              HTTP,DNS -DnsVhost.py              #
###################################################
## 1. http, DNS 설치 및 초기 세팅 
def install_and_setting_http_dns() :
    #========= HTTP ========#
    cc.run(dv.install_http())
    dv.set_servername()
    cc.run(dv.mk_vhostconf())
    cc.run(dv.test_vhost())
    # web_dir,third_domain,error_log ,custom_log
    cc.run(dv.set_vhostconf("/home/test","test","/var/log/httpd/test_error.log","/var/log/httpd/test_access.log"))

    #========= DNS ========#
    cc.run(dv.install_named())
    cc.run(dv.init_set_named_conf())
    cc.run(dv.init_set_named_rfc())
    cc.run(dv.mk_zone_file())
    cc.run(dv.init_set_zone_file())
    cc.run(dv.set_dns_Third_domain("test"))

    #======= RESTART ========#
    cc.run(dv.restart_http())
    cc.run(dv.restart_dns())
    
## 2. 초기 세팅 이후에 새롭게 DnsVhost클래스 사용할 경우에 해당됨 (기본)
def mod_http_dns(web_dir,third_domain,error_log ,custom_log):
    dv.set_servername()
    #set_vhostconf(self,web_dir,third_domain,error_log ,custom_log):
    cc.run(dv.set_vhostconf(web_dir,third_domain,error_log ,custom_log))
    cc.run(f"mkdir -p {web_dir} && chmod 755 {web_dir}")
    cc.run(dv.set_dns_Third_domain(third_domain))
    cc.run(dv.restart_dns())
    cc.run(dv.restart_http())

## 2. 초기 세팅 이후에 새롭게 DnsVhost클래스 사용할 경우에 해당됨 (리버시 프록시)
def mod_http_dns_reverse_proxy(third_domain,error_log ,custom_log):
    dv.set_servername()
    #set_reverse_proxy_vhostconf(self,third_domain,error_log ,custom_log)
    cc.run(dv.set_reverse_proxy_vhostconf(third_domain,error_log ,custom_log))
    cc.run(dv.set_dns_Third_domain(third_domain))
    cc.run(dv.restart_dns())
    cc.run(dv.restart_http())
#---------------------실행부-----------------------#
#install_and_setting_http_dns()
#mod_http_dns("/home/test2/","test2","/var/log/httpd/test2_error.log","/var/log/httpd/test2_access.log")



###################################################
#            MariaDB,PHP -MariaDB.py              #
###################################################
## 1. mariaDB와 php 둘 다 설치함
def install_mariaDB_and_php():
    cc.run(mariadb.install_packages())
    cc.run(mariadb.install_apache_and_php())
    cc.run(mariadb.install_mariaDB())
    cc.run(mariadb.init_mariaDB())
    cc.run(mariadb.install_php_my_admin())
    cc.run(mariadb.set_phpMyadmin_conf())
#---------------------실행부-----------------------#
#install_mariaDB_and_php()


##################################################
#           Partition -Partition.py              #
##################################################
def mk_partition():
    cc.run("lsblk")
    cc.run(part.make_partition())
    cc.run(part.format_partition())
    cc.run(part.mount())
    cc.run(part.permanent_mount_config_remote())


#---------------------실행부-----------------------#
#mk_partition()

###################################################
#                    SMB -SMB.py                  #
###################################################

def install_smb():
    cc.run(smb.smb_install())
    cc.run(smb.smb_intsll_status_check())   
    choose = input("smb 서버와 클라이언트 중 선택하세요.(S/C) ex)S : ")
    if(choose == "S" ) :
        server_smb()
    if(choose == "C" ) :
        client_smb()

def server_smb():
    cc.run(smb.smb_share_folder_make())
    cc.run(smb.smb_mk_smb_id())
    cc.run(smb.smb_set_smb_id_passwd())
    cc.run(smb.smb_id_check())
    cc.run(smb.smb_share_folder_authority())
    cc.run(smb.smb_mk_back())
    cc.run(smb.smb_set())
    cc.run(smb.smb_server_start())
    
def client_smb() :
    cc.run(smb.smb_client_login())
    cc.run(smb.smb_client_mount())
#---------------------실행부-----------------------#
#install_smb()

###################################################
#                    NFS -NFS.py                  #
###################################################

def install_nfs():
    cc.run(nfs.nfs_isntall())
    choose = input("NFS 서버와 클라이언트 중 선택하세요.(S/C) ex)S : ")
    if(choose == "S" ) :
        server_nfs()
    if(choose == "C" ) :
        client_nfs()

def server_nfs():
    cc.run(nfs.nfs_share_folder_make())
    cc.run(nfs.nfs_share_folder_authority())
    cc.run(nfs.server_nfs_set_exports_config())
    cc.run(nfs.server_nfs_start())

def client_nfs():
    cc.run(nfs.nfs_client_mount_temporary())
    cc.run(nfs.nfs_clinet_mount_permanent())
#---------------------실행부-----------------------#
#install_nfs()

###################################################
#            Diskquota -diskquota.py              #
###################################################
def diskquota_install_and_set():
    cc.run(dq.diskquota_isntall())
    cc.run(dq.diskquota_install_check())
    cc.run("lsblk")
    print("TYPE MOUNTPOINTS가 part로 되어있는 디스크만 가능합니다. 안되면 partition을 해야합니다")
    exist_part = input("part로 되어있는 디스크가 없으시거나 해당 디스크가 마운트가 안되어있는 경우에 반드시 n 입력해주세요.\n 있으시면 y 또는 아무문자열 입력 ex) y: ")
    if(exist_part == "n") :
        return
    cc.run(dq.diskquota_modify())
    cc.run(dq.diskquota_mount_and_initalization())
    cc.run(dq.diskquota_settings())
    cc.run(dq.diskquota_systemctl())
    
#---------------------실행부-----------------------#
#diskquota_install_and_set()



###################################################
#                  ISCSI -ISCSI.py                #
###################################################
def iscsi_all_in_one():

    #=========== server 영역 ===========#
    #cc.run(iscsi.set_firewall())
    cc.run(iscsi.install_target())
    
    #=========== client 영역 ===========#
    client_ip = input("새로운 SSH 연결을 합니다. 클라이언트 IP를 입력하세요")
    client_account = input("클라이언트 계정 입력하세요 ex. root")
    client_passwd = input("클라이언트 계정의 비밀번호를 입력하세요 ex. asd123!@")
    cc_client =  CC(f"{client_ip}", f"{client_account}", f"{client_passwd}")
    cc_client.connect()
    
    cc_client.run(iscsi.iscsi_client_install())
    cc_client.run(iscsi.iscsi_client_iqn())
    iscsi_client_iqn_domain = input("위에서 확인 CLIENT의 IQN을 복사해주세요")

    #=========== server 영역 ===========#
    print("[ISCSI 서버] ISCSI 서버를 세팅해보겠습니다.")
    cc.run("lsblk")
    disk_path = input("위의 결과를 확인하시고 iSCSI로 제공할 디스크 경로를 입력하세요 : ex)/dev/sdb")
    cc.run(iscsi.targetcli(server_ip, iscsi_client_iqn_domain, disk_path))

    #=========== client 영역 ===========#
    cc_client.run(iscsi.iscsi_client_login_set_servers_iscsi(server_ip))
    cc_client.run("iscsiadm -m session")
    print("이제 클라이언트로 돌아가셔서 lsblk로 확인해보세요. 또 iscsiadm -m session로 잘 받았는지 확인해보세요 ^^")
#---------------------실행부-----------------------#
#iscsi_all_in_one()

###################################################
#                    FTP -FTP.py                  #
###################################################
def inital_FTP():
    print("[FTP 서버] FTP 설치 및 유저를 생성합니다.")
    cc.run(ftp.install_ftp())
    cc.run(ftp.on_off_ftp())
    cc.run(ftp.set_ftp_config())
    cc.run(ftp.add_ftpuser())
    cc.run(ftp.set_ftpuser_passwd())
    cc.run(ftp.set_dir())
    
#---------------------실행부-----------------------#
#inital_FTP()

###################################################
#                  RAID -RAID.py                  #
###################################################
def mk_raid1():
    print("[RAID] RAID1을 만듭니다.")
    cc.run(raid.install_raid())
    cc.run("lsblk")
    cc.run(raid.raid1_mk())
    cc.run(raid.raid1_mk_fs_mount())
    permanent_mount = input("영구마운트 하겠습니까 yes or no (ex. yes) : ")    
    if(permanent_mount == "yes") :
        cc.run(raid.raid1_permanent_mount_config_remote())
    print("완료 했습니다!")
    cc.run("lsblk")
    
def erase_raid():
    print("[RAID] RAID를 제거합니다.")
    cc.run("lsblk")
    cc.run(raid.erase_raid())
    cc.run("lsblk")
    print("RAID가 제거되었습니다 ^^")
    
#---------------------실행부-----------------------#
#mk_raid1()
#erase_raid()


###################################################
#               Tomcat -Tomcat.py                 #
###################################################
def install_tomcat():
    print("[Tomcat] 설치하겠습니다.")
    cc.run(tomcat.install_java())
    cc.run(tomcat.set_tomcat())
  #  mod_http_dns_reverse_proxy(web_dir,third_domain,error_log ,custom_log):
    mod_http_dns_reverse_proxy("tom","/var/log/httpd/tom_error_log", "/var/log/httpd/tom_access_log")
    cc.run(tomcat.set_service())
    

#---------------------실행부-----------------------#
install_tomcat()

###################################################
#                 WP,PHPMYSQL -WP.py              #
###################################################
## 1. 로컬) WP 설치 및 wp-config.php 파일 수정까지
def install_wp():
    wp.install_prev_packages()
    wp.install_and_move_dir()
    wp.set_wp_config_p()

실행
MariaDB init start
[Tomcat] 설치하겠습니다.
--2025-12-27 00:47:34--  https://download.oracle.com/java/25/latest/jdk-25_linux-x64_bin.rpm
Resolving download.oracle.com (download.oracle.com)... 104.92.144.85
Connecting to download.oracle.com (download.oracle.com)|104.92.144.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 214998746 (205M) [application/x-redhat-package-manager]
Saving to: ‘jdk-25_linux-x64_bin.rpm.1’

jdk-25_linux-x64_bi 100%[===================>] 205.04M  10.7MB/s    in 20s     

2025-12-27 00:47:54 (10.5 MB/s) - ‘jdk-25_linux-x64_bin.rpm.1’ saved [214998746/214998746]

Last metadata expiration check: 0:06:11 ago on Sat Dec 27 00:41:45 2025.
Package jdk-25-2000:25.0.1-8.x86_64 is already installed.
Dependencies resolved.
Nothing to do.
Complete!
Last metadata expiration check: 0:06:12 ago on Sat Dec 27 00:41:45 2025.
Package unzip-6.0-59.el9.x86_64 is already installed.
Dependencies resolved.
Nothing to do.
Complete!

useradd: user 'tomcat' alread

ServerName을 입력하세요. (예: test.com , ppp.com):  test.com


bash: line 10: warning: here-document at line 2 delimited by end-of-file (wanted `EOF')



사용할 DNS의 ip:  172.16.16.25




httpd restart 시작

Created symlink /etc/systemd/system/multi-user.target.wants/tomcat.service → /etc/systemd/system/tomcat.service.

